#### Pipeline

0) prepare data
1) Design model (input, output, forward pass with different layers)
2) Construct loss and optimizer
3) Training loop
      - Forward = compute prediction and loss
      - Backward = compute gradients
      - Update weights

In [13]:
import torch
import torch.nn as nn
import numpy as np
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

0. Prepare the data

In [16]:
# prepare data
bc = datasets.load_breast_cancer()
X, y= bc.data, bc.target
print(X.shape)
n_samples, n_features=X.shape
X_train,X_test, y_train, y_test=train_test_split(X,y, test_size=0.2, random_state=1234)

#scale
sc = StandardScaler()   # Standardize features by removing the mean and scaling to unit variance.
X_train = sc.fit_transform(X_train) # scaling the data
X_test = sc.fit_transform(X_test)

# convert to tensor
X_train = torch.from_numpy(X_train.astype(np.float32))
X_test= torch.from_numpy(X_test.astype(np.float32))
y_train = torch.from_numpy(y_train.astype(np.float32))
y_test= torch.from_numpy(y_test.astype(np.float32))

# Reshape the target column
y_train= y_train.view(y_train.shape[0],1)
y_test= y_test.view(y_test.shape[0],1)
print(y_test.shape)

(569, 30)
torch.Size([114, 1])


1. Model Creation

In [20]:
## 1. Model
class Model(nn.Module):
    def __init__(self, n_input_features):
        super(Model, self).__init__()
        '''
        Applies a linear transformation to the incoming data: :math:`y = xA^T + b`, arg
        Args:
        in_features: size of each input sample
        out_features: size of each output sample
        bias: If set to ``False``, the layer will not learn an additive bias.
            Default: ``True`` 
        '''
        self.linear = nn.Linear(n_input_features,1)
    
    '''
    prediction
    forward(self, input)
    '''
    def forward(self, x):
        y_pred= torch.sigmoid(self.linear(x))
        return y_pred

2. Loss and Optimizer

In [25]:
model = Model(n_features)
#2 Loss and Optimizer
num_epochs = 200
learning_rate = .01
criterian = nn.BCELoss()  # measures the Binary Cross Entropy
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

3. Training Loop 

In [27]:
# Traing loop
for epoch in range(num_epochs):
    # forward pass and loss
    y_pred=model(X_train)
    loss= criterian(y_pred,y_train)

    # backward pass
    loss.backward()
    optimizer.step()

    optimizer.zero_grad()
    if(epoch+1)%10==0:
         print(f'epoch: {epoch+1}, loss = {loss.item():.4f}')

with torch.no_grad():
     y_predicted = model(X_test)
     y_predicted_cls = y_predicted.round()
     acc = y_predicted_cls.eq(y_test).sum()/float(y_test.shape[0])
     print(f'accuracy: {acc.item():.4f}')

epoch: 10, loss = 0.1768
epoch: 20, loss = 0.1728
epoch: 30, loss = 0.1692
epoch: 40, loss = 0.1657
epoch: 50, loss = 0.1625
epoch: 60, loss = 0.1595
epoch: 70, loss = 0.1566
epoch: 80, loss = 0.1540
epoch: 90, loss = 0.1514
epoch: 100, loss = 0.1490
epoch: 110, loss = 0.1467
epoch: 120, loss = 0.1446
epoch: 130, loss = 0.1425
epoch: 140, loss = 0.1405
epoch: 150, loss = 0.1387
epoch: 160, loss = 0.1369
epoch: 170, loss = 0.1352
epoch: 180, loss = 0.1335
epoch: 190, loss = 0.1320
epoch: 200, loss = 0.1304
accuracy: 0.9211
